In [12]:
import warnings
warnings.filterwarnings('ignore', message='Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.')

from pynq import Overlay, MMIO
# 初始化硬體
overlay = Overlay('./hw10.bit')

# 設定記憶體位址
cdma_base = overlay.ip_dict['axi_cdma_0']['phys_addr']
gpio_base = overlay.ip_dict['axi_gpio_0']['phys_addr']
zynq_addr = 0x30000000
bram0_addr = 0xC0000000
bram1_addr = 0xC2000000

# 初始化MMIO物件
zynq_sys = MMIO(zynq_addr, 0x1500)
cdma_ctrl = MMIO(cdma_base + 0x00, 0x18)
cdma_sa = MMIO(cdma_base + 0x18, 0x1500)
cdma_da = MMIO(cdma_base + 0x20, 0x1500)
cdma_btt = MMIO(cdma_base + 0x28, 0x10)
gpio_a = MMIO(gpio_base + 8, 8)

# 載入輸入資料
print('start initial!\n')
offset = 0x0
with open('input.hex', "r") as f_in:
    for line in f_in:
        if not line:
            break
        zynq_sys.write(offset, int('0x' + line.strip(), 16))
        offset += 4

# 寫入特定資料
specific_data = [
    (784, 0x0000A89E), (785, 0x000092D5), (786, 0x00006D43), (787, 0x00001004),
    (788, 0xFFFF8F71), (789, 0xFFFF6E54), (790, 0xFFFFA6D7), (791, 0xFFFFC834),
    (792, 0xFFFFAC19), (793, 0x0)
]
for addr_offset, data in specific_data:
    zynq_sys.write(addr_offset * 4, data)

# 執行CDMA傳輸和處理
cdma_ctrl.write(0x0, 0x04)
cdma_sa.write(0x0, zynq_addr)
cdma_da.write(0x0, bram0_addr)
cdma_btt.write(0x0, 0x1500)
print('initial finish!\n')

# GPIO控制
gpio_a.write(0x4, 0)
gpio_a.write(0, 1)
for _ in range(100000):
    pass
gpio_a.write(0x4, 0)
gpio_a.write(0, 0)

# 讀取輸出
cdma_ctrl.write(0x0, 0x04)
cdma_sa.write(0x0, bram1_addr)
cdma_da.write(0x0, zynq_addr)
cdma_btt.write(0x0, 0x1500)

# 儲存輸出資料
offset = 0x0
with open('output.hex', 'w') as f_out:
    while offset < 676 * 4:
        raw_data = zynq_sys.read(offset)
        hex_str = str(hex(raw_data)).upper()[2:]
        if len(hex_str) != 8:
            hex_str = '0' * (8 - len(hex_str)) + hex_str
        f_out.write(hex_str + '\n')
        offset += 4

print('all finish!')

# 比較輸出與黃金檔案
golden_file = open('golden.hex', 'r')
g_lines = golden_file.readlines()
err = 0
addr = 0

for line in g_lines: 
    expect = hex(int(line[0:8], 16))
    # Read from memory
    output = hex(zynq_sys.read(addr))
    if(output != expect):
        print(f"[Error] Output is {output} expected {expect} at addr {addr}")
        err += 1
    addr += 4

if(err == 0):
    print("Congradulation ! ALL pass")
else:
    print(f"Error ! data mismatch ! Totally {err} errors")

golden_file.close()

start initial!

initial finish!

all finish!
Congradulation ! ALL pass
